Key, tempo(BPM)

In [ ]:
import os
import json
from essentia.standard import MonoLoader, KeyExtractor, RhythmExtractor2013

# Define the directory containing the audio files
directory_path = "/content/processed"  # Replace with your directory path(in .wav files)

# List to store results
results = []

# Iterate through all files in the directory
for file_name in os.listdir(directory_path):
    if file_name.endswith(".wav") or file_name.endswith(".mp3"):
        file_path = os.path.join(directory_path, file_name)
        print(f"Processing: {file_name}")

        # Load the audio file
        loader = MonoLoader(filename=file_path)
        audio = loader()

        # Extract the key
        key_extractor = KeyExtractor()
        key, scale, strength = key_extractor(audio)

        # Extract the tempo (BPM)
        rhythm_extractor = RhythmExtractor2013(method="multifeature")
        bpm, confidence, _, _, _ = rhythm_extractor(audio)

        # Convert bpm to scalar (if it's a numpy array)
        bpm = float(bpm)

        # Append the result as a dictionary
        results.append({
            "file_name": file_name,
            "key": key,
            "scale": scale,
            "strength": round(strength, 2),
            "tempo_bpm": round(bpm, 2)
        })

# Print results
print("\nFinal Results in JSON Format:")
print(json.dumps(results, indent=4))

# Optionally save the results to a JSON file
output_path = os.path.join(directory_path, "results.json")
with open(output_path, "w") as json_file:
    json.dump(results, json_file, indent=4)

print(f"\nResults saved to: {output_path}")


MIDI

In [ ]:
import os
import shutil
from basic_pitch.inference import predict_and_save
from basic_pitch import ICASSP_2022_MODEL_PATH

# Define the base directory containing the stems
base_dir = '/content/5' # Adjust to your actual path where stems are present

# Function to process each audio file
def process_audio_file(audio_path, output_dir):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    # Perform prediction and save the MIDI file
    predict_and_save(
        audio_path_list=[audio_path],  # Corrected argument name
        output_directory=output_dir,
        save_midi=True,
        sonify_midi=False,
        save_model_outputs=False,
        save_notes=False,
        model_or_model_path=ICASSP_2022_MODEL_PATH
    )

# Traverse the directory structure
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith('.wav'):
            audio_path = os.path.join(root, file)
            # Define the corresponding output directory for MIDI files
            relative_path = os.path.relpath(root, base_dir)
            output_dir = os.path.join('/content/o-5', relative_path)  # Adjust output path as needed
            process_audio_file(audio_path, output_dir)

# Define the output directory and ZIP file path
output_dir = '/content/o-5'
zip_file_path = '/mnt/data/5.zip'

# Check if the output directory exists
if os.path.exists(output_dir):
    # Create a ZIP file of the directory
    shutil.make_archive(base_name=zip_file_path.replace('.zip', ''), format='zip', root_dir=output_dir)
    print(f"Output directory zipped successfully. Download the ZIP file from {zip_file_path}")
else:
    print(f"Output directory '{output_dir}' does not exist. Ensure the processing step is completed.")
